In [29]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yfinance as yf
import numpy as np
import talib as ta

In [30]:
%matplotlib inline
sns.set_theme(rc={"figure.figsize" : [20,5]},
              style='dark')
#sns.set_style("dark")
sns.set_context(context='notebook')
plt.style.use('dark_background')

In [31]:
activo = 'LINK-USD'
df = yf.Ticker(activo)
df = df.history(start='2018-01-01', end='2024-11-28')

In [32]:
#Definir función para calcular el rendimiento de la estrategia
def backtest_strategy(short_window, long_window):
    df['EMA_short'] = ta.EMA(df['Close'], timeperiod=short_window)
    df['EMA_long'] = ta.EMA(df['Close'], timeperiod=long_window)
    df['Signal'] = np.where(df['EMA_short'] > df['EMA_long'], 1, -1)
    df['Position'] = df['Signal'].shift()
    df['Strategy_Returns'] = df['Position'] * df['Close'].pct_change()
    return (df['Strategy_Returns'] + 1).cumprod().iloc[-1] -1 # Retorno total

In [33]:
# Definir el rango de parámetros a probar
short_windows = range(2, 100, 1) # De 10 a 100 días 
long_windows = range(2, 200, 1) # de 10 a 200 días

In [34]:
# probar todas las combinaciones de parámetros
best_performance = -np.inf
best_params = (None, None)

In [35]:
for short_window in short_windows:
    for long_window in long_windows:
        if short_window >= long_window:
            continue  # Evitar combinaciones donde la media corta sea mayor o igual a la larga
        performance = backtest_strategy(short_window, long_window)
        if performance > best_performance:
            best_performance = performance
            best_params = (short_window, long_window)

print(f"Mejores parámetros: SMA corta={best_params[0]}, SMA larga={best_params[1]}")
print(f"Mejor rendimiento: {best_performance:.2%}")

Mejores parámetros: SMA corta=5, SMA larga=6
Mejor rendimiento: 1143.93%
